<a href="https://colab.research.google.com/github/abhinavkumar17/Machine-Learnings/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tflearn

     |████████████████████████████████| 112kB 9.0MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp36-none-any.whl size=127301 sha256=6f4facf54060dc7ea3f10fb6e085b3ee54c27b1de4ac488c7e9110d619d78684
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [2]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [4]:
nltk.download('punkt')
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [5]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
39 documents
14 classes ['SyncUP PETS App', 'device specifications', 'goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'pair', 'payments', 'pets app Requirements', 'rental', 'syncup pets', 'thanks', 'today']
61 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ap', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'dev', 'do', 'doe', 'get', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'in', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mobl', 'mop', 'of', 'on', 'op', 'pair', 'pet', 'rent', 'requir', 'see', 'spec', 'syncup', 'tak', 'thank', 'that', 'the', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'wher', 'which', 'with', 'work', 'yo', 'you']


In [6]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
39 documents
14 classes ['SyncUP PETS App', 'device specifications', 'goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'pair', 'payments', 'pets app Requirements', 'rental', 'syncup pets', 'thanks', 'today']
61 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ap', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'dev', 'do', 'doe', 'get', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'in', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mobl', 'mop', 'of', 'on', 'op', 'pair', 'pet', 'rent', 'requir', 'see', 'spec', 'syncup', 'tak', 'thank', 'that', 'the', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'wher', 'which', 'with', 'work', 'yo', 'you']


In [7]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [8]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")
#tf.reset_default_graph()

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [9]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training....


In [10]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [11]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.48309 | time: 0.019s
| Adam | epoch: 1000 | loss: 0.48309 - acc: 0.9605 -- iter: 32/39
Training Step: 5000  | total loss: 0.43610 | time: 0.025s
| Adam | epoch: 1000 | loss: 0.43610 - acc: 0.9644 -- iter: 39/39
--
Saving the Model.......
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [13]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [14]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer
    print("\n")

Battery
• Battery Type: Li-Ion
• Battery capacity (mAh): 430
• Battery Charger: Dock with pogo pins, USB-C to A cable, USB-A AC adapter
• Expected Battery Life: Up to 8 days depending on usage and ability to use WiFi and BLE connections
• Highest specified charging temperature: 45℃
• Lowest specified charging temperature: 11℃
Note: The device may not charge at high temperatures (>40℃/104F) to prevent
permanent damage to the battery and tracker.
General
• Display: None
• Waterproof/Dust Rating: IP67
• SIM: Embedded
• Sensors: Accelerometer, Temperature
• User Controls: Button
• LED Light: Red, Green, Blue, White (RGBW)
• Speaker: Piezoelectric buzzer

SyncUP PETS User Guide 10
Networks
• NB-IOT
• LTE Bands: 2, 4, 12, and 66
• Roaming: Not supported
• FCC ID: 2ASXC-TMO-NBT-01
Connectivity
• Wi-Fi: 802.11n on 2.4GHz
• Bluetooth® BLE version 4.2 Class 1




In [ ]:
from google.colab import drive
drive.mount('/content/drive')